In [1]:
import os
os.chdir('D:\Master_Code_92589')

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import euclidean_distances
import statistics
import pandas as pd 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import csv
from sklearn import tree
#import xgboost as xgb
from datetime import datetime
import pickle
from random import seed
from random import random

table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

def coor_to_class(s):
    return table[eval(s)[1]][eval(s)[0]]

def to_int(s):
    return int(s)

def class_to_coordinate(a):
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    coordinate = [x,y]
    return coordinate

def euclidean_distance(p1,p2):
    return np.sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]))

def uuid_to_b_num(s):
    if s[0:30] == 'e20a39f4-73f5-4bc4-a12f-17d1ad' :
        uuid_num = int(s[-1])
    else :
        uuid_num = 0
    return uuid_num

## 說明：
- 經由 app 收集而來的 BLE_data.csv 把每 10 秒收到的同個 beacon 的 RSSI 做平均產生 wireless fingerprint，每次一共會產生四個檔案，
- 在這一階段可以操作每台手機的 packet reception rate，以及使用的 beacon 數量
- 以下列檔案為例，"avg_10" 表示取 10 秒的 RSSI 做平均，"rate_7" 表示 packet reception rate=0.7
    1. raw_fingerprint_avg_10_rate_7.csv: 不做平均的 fingerprint，每個 step 可能會有多個 fingerprint，之後會用這個檔案來算真實的 packet reception rate，詳見 "Packet_loss_rate.ipynb"

    2. wireless_fingerprint_avg_10_**6_beacon**_rate_7.csv: 使用 6 個 beacon 的情況產生的 fingerprint

    3. wireless_fingerprint_avg_10_**4_beacon**_rate_7.csv: 使用 4 個 beacon 的情況產生的 fingerprint

    4. wireless_fingerprint_avg_10_**2_beacon**_rate_7.csv: 使用 2 個 beacon 的情況產生的 fingerprint


## 參數：
- loss_rate: packet reception rate, 要保留多少比例的封包，例如：loss_rate=0.7，表示會隨機忽略30%的封包

- sec: 一次要抓幾秒來算平均，因為我們收資料時每10秒走一步，所以設定一次抓10秒算平均

In [3]:
# Input
loss_rate = 1.0
sec = 10
walks = ['stationary_1',\
        'freewalk_1','freewalk_2','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_7','freewalk_8','freewalk_9',\
        'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
        'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
        ]
phones = ['GalaxyA51','hTCU11','hTCU19e','sharp025']

In [4]:
with open(f'./Data/0318_92589_train/min_rssi.pickle', 'rb') as f:
    min_rssi = pickle.load(f)
with open(f'./Data/0318_92589_train/max_rssi.pickle', 'rb') as f:
    max_rssi = pickle.load(f)

for walk in walks:

    test_dir = f'./Data/0318_92589_test'

    for phone in phones:

        wireless_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
        raw_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])

        start_end = pd.read_csv(f'{test_dir}/{walk}.csv', delimiter=',')
        start_end = start_end[start_end['phone'] == phone]

        BLE_df = pd.read_csv(f'{test_dir}/{walk}/{phone}/BLE_data.csv', delimiter=',')
        sensor_data = pd.read_csv(f'{test_dir}/{walk}/{phone}/sensor_data.csv', delimiter=',')

        # print(start_end)
        Min = min_rssi[phone]
        Max = max_rssi[phone]

        for idx, Row in start_end.iterrows(): # for every step

            time = sensor_data.copy()
            df = BLE_df.copy()

            avg_rssi = (df['RSSI'].mean()-Min)/(Max-Min)
            # print(avg_rssi)

            x = int(Row['position']/100)
            y = int(Row['position']%100)
            position = table[y][x]

            start_sec = Row['start']
            # print(start_sec)
            time['Time'] = pd.DatetimeIndex(time['Time'])
            init_time = time.iloc[0]['Time']
            start_time = init_time + pd.to_timedelta(start_sec, unit='s')
            end_time = start_time + pd.to_timedelta(sec, unit='s') # add 10 second

            df['Time'] = pd.DatetimeIndex(df['Time'])
            df.set_index(keys='Time', inplace=True)
            df = df.between_time(str(start_time)[11:], str(end_time)[11:])
            df = df.reset_index()
            df['UUID'] = df['UUID'].apply(uuid_to_b_num)

            # 整合為 beacon 形式
            rssi_time = []
            rssi = [np.zeros((8))]
            rssi[0][:] = np.nan 
            rssi[0][0] = position
            row = 0
            for i in range(len(df) - 1):
                curr = df.iloc[i]['Time']
                next = df.iloc[i+1]['Time']

                rssi_idx = int(df.iloc[i]['UUID'])

                if random() <= loss_rate:
                    rssi[row][rssi_idx] = (df.iloc[i]['RSSI']-Min)/(Max-Min)

                if curr != next:  
                    rssi_time.append(curr)  
                    row += 1
                    rssi.append(np.zeros((8)))
                    rssi[row][:] = np.nan
                    rssi[row][0] = position

            rssi_time.append(df.iloc[len(df) - 1]['Time'])
            rssi_idx = int(df.iloc[len(df) - 1]['UUID'])

            if random() <= loss_rate:
                rssi[row][rssi_idx] = (df.iloc[len(df) - 1]['RSSI']-Min)/(Max-Min)

            test_set = pd.DataFrame(np.array(rssi), columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
            test_set['Time'] = rssi_time
            
            # 取該段時間平均RSSI
            tmp = {'label':[position]} 
            for column in test_set.columns[1:]:
                tmp[column] = [test_set[column].mean()]

            testing_data = pd.DataFrame(tmp,columns=test_set.columns)

            wireless_fingerprint = wireless_fingerprint.append(testing_data,ignore_index=True)
            raw_fingerprint = raw_fingerprint.append(test_set,ignore_index=True)

            wireless_fingerprint = wireless_fingerprint.interpolate(limit_direction='both')

        print(f'walk:{walk},phone:{phone},wireless_fingerprint:\n{wireless_fingerprint},raw_fingerprint:\n{raw_fingerprint}')

        wireless_fingerprint = wireless_fingerprint.drop(['Time'], axis='columns')
        wireless_5_beacon = wireless_fingerprint.drop(['Beacon_3','Beacon_4'], axis='columns')
        wireless_4_beacon = wireless_fingerprint.drop(['Beacon_3','Beacon_4','Beacon_7'], axis='columns')

        wireless_fingerprint.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_7_beacon_rate_{int(10*loss_rate)}.csv')
        
        wireless_5_beacon.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_5_beacon_rate_{int(10*loss_rate)}.csv')
        wireless_4_beacon.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_4_beacon_rate_{int(10*loss_rate)}.csv')

        raw_fingerprint.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/raw_fingerprint_avg_{sec}_rate_{int(10*loss_rate)}.csv')

walk:stationary_1,phone:GalaxyA51,wireless_fingerprint:
  label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  Beacon_7  \
0    21  0.260870  0.358696  0.355072  0.413043  0.485507  0.782609  0.471014   
1    41  0.583333  0.467391  0.507246  0.510870  0.608696  0.420290  0.423913   
2    36  0.507246  0.400966  0.661836  0.415459  0.454106  0.425121  0.690821   
3    16  0.311594  0.579710  0.492754  0.500000  0.478261  0.384058  0.471014   
4    11  0.454106  0.405797  0.589372  0.652174  0.304348  0.429952  0.690821   
5     1  0.541063  0.632850  0.381643  0.516908  0.338164  0.362319  0.454106   
6    31  0.550725  0.521739  0.514493  0.384058  0.304348  0.297101  0.481884   

                           Time  
0 2022-03-21 00:14:28.749999872  
1 2022-03-21 00:15:27.750000128  
2 2022-03-21 00:16:04.000000000  
3 2022-03-21 00:16:39.000000000  
4 2022-03-21 00:17:22.000000000  
5 2022-03-21 00:17:58.999999744  
6 2022-03-21 00:18:37.500000000  ,raw_fingerprint:
    la

walk:stationary_1,phone:sharp025,wireless_fingerprint:
  label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  Beacon_7  \
0    21  0.394737  0.377193  0.350877  0.517544  0.254386  0.798246  0.324561   
1    41  0.500000  0.342105  0.622807  0.412281  0.631579  0.333333  0.482456   
2    36  0.397661  0.421053  0.672515  0.421053  0.421053  0.438596  0.520468   
3    16  0.461988  0.473684  0.526316  0.573099  0.380117  0.385965  0.777778   
4    11  0.473684  0.508772  0.535088  0.771930  0.368421  0.491228  0.640351   
5     1  0.473684  0.771930  0.447368  0.289474  0.228070  0.421053  0.456140   
6    31  0.719298  0.552632  0.403509  0.394737  0.280702  0.350877  0.421053   

                           Time  
0 2022-03-21 00:14:25.500000000  
1 2022-03-21 00:15:26.500000000  
2 2022-03-21 00:16:00.333333248  
3 2022-03-21 00:16:37.000000256  
4 2022-03-21 00:17:20.500000000  
5 2022-03-21 00:17:56.500000000  
6 2022-03-21 00:18:36.500000000  ,raw_fingerprint:
    lab

walk:freewalk_1,phone:hTCU19e,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0     12  0.516667  0.683333  0.666667  0.783333  0.333333  0.491667   
1     15  0.475000  0.566667  0.525000  0.683333  0.483333  0.466667   
2     18  0.616667  0.450000  0.683333  0.650000  0.366667  0.241667   
3     36  0.491667  0.450000  0.783333  0.583333  0.358333  0.575000   
4     36  0.616667  0.458333  0.733333  0.650000  0.441667  0.450000   
5     33  0.616667  0.566667  0.750000  0.383333  0.491667  0.291667   
6     31  0.700000  0.516667  0.600000  0.316667  0.441667  0.358333   
7     31  0.741667  0.450000  0.600000  0.350000  0.316667  0.358333   
8     34  0.775000  0.416667  0.766667  0.458333  0.491667  0.466667   
9     36  0.791667  0.375000  0.841667  0.508333  0.433333  0.500000   
10    39  0.716667  0.441667  0.800000  0.508333  0.550000  0.558333   
11    41  0.725000  0.408333  0.666667  0.458333  0.800000  0.583333   
12    41  0.

walk:freewalk_2,phone:GalaxyA51,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0      2  0.594203  0.586957  0.391304  0.434783  0.434783  0.376812   
1      5  0.545894  0.512077  0.391304  0.381643  0.260870  0.381643   
2      8  0.594203  0.444444  0.381643  0.362319  0.260870  0.289855   
3     31  0.710145  0.463768  0.471014  0.304348  0.376812  0.347826   
4     31  0.673913  0.521739  0.597826  0.300725  0.463768  0.336957   
5     34  0.557971  0.355072  0.615942  0.380435  0.355072  0.398551   
6     36  0.528986  0.398551  0.518116  0.514493  0.387681  0.340580   
7     36  0.512077  0.342995  0.661836  0.473430  0.381643  0.352657   
8     33  0.601449  0.514493  0.478261  0.376812  0.347826  0.246377   
9     31  0.623188  0.514493  0.351449  0.445652  0.297101  0.304348   
10    31  0.608696  0.496377  0.550725  0.391304  0.449275  0.434783   
11    34  0.485507  0.322464  0.615942  0.384058  0.289855  0.405797   
12    36  

walk:freewalk_2,phone:hTCU11,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0      2  0.492308  0.646154  0.400000  0.507692  0.430769  0.430769   
1      5  0.353846  0.671795  0.471795  0.492308  0.343590  0.364103   
2      8  0.561538  0.553846  0.446154  0.392308  0.315385  0.461538   
3     31  0.738462  0.523077  0.515385  0.553846  0.192308  0.430769   
4     31  0.687179  0.538462  0.451282  0.466667  0.369231  0.446154   
5     34  0.630769  0.500000  0.584615  0.361538  0.330769  0.407692   
6     36  0.746154  0.438462  0.615385  0.553846  0.500000  0.369231   
7     36  0.569231  0.492308  0.753846  0.500000  0.446154  0.476923   
8     33  0.776923  0.492308  0.630769  0.515385  0.369231  0.484615   
9     31  0.676923  0.615385  0.461538  0.553846  0.469231  0.407692   
10    31  0.707692  0.553846  0.584615  0.492308  0.476923  0.461538   
11    34  0.684615  0.492308  0.523077  0.323077  0.369231  0.384615   
12    36  0.7

...

walk:round_trip_walk_2,phone:sharp025,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0      1  0.456140  0.771930  0.391813  0.578947  0.198830  0.345029   
1      2  0.342105  0.649123  0.438596  0.464912  0.140351  0.315789   
2      3  0.535088  0.719298  0.342105  0.385965  0.263158  0.394737   
3      4  0.473684  0.666667  0.324561  0.543860  0.175439  0.368421   
4      5  0.350877  0.622807  0.385965  0.473684  0.236842  0.421053   
..   ...       ...       ...       ...       ...       ...       ...   
61     5  0.543860  0.622807  0.324561  0.526316  0.210526  0.438596   
62     4  0.614035  0.614035  0.321637  0.380117  0.228070  0.345029   
63     3  0.596491  0.666667  0.271930  0.491228  0.271930  0.201754   
64     2  0.321637  0.631579  0.438596  0.385965  0.269006  0.309942   
65     1  0.500000  0.798246  0.482456  0.421053  0.245614  0.385965   

    Beacon_7                          Time  
0   0.561404 2022-03-20 20:26:

walk:round_trip_walk_3,phone:hTCU11,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0      1  0.338462  0.792308  0.576923  0.323077  0.300000  0.561538   
1      2  0.538462  0.589744  0.492308  0.528205  0.297436  0.328205   
2      3  0.589744  0.553846  0.553846  0.528205  0.317949  0.348718   
3      4  0.461538  0.638462  0.461538  0.492308  0.276923  0.376923   
4      5  0.430769  0.569231  0.484615  0.507692  0.276923  0.438462   
5      6  0.576923  0.592308  0.215385  0.530769  0.276923  0.338462   
6      7  0.553846  0.561538  0.446154  0.438462  0.276923  0.446154   
7      8  0.538462  0.700000  0.438462  0.592308  0.284615  0.492308   
8      9  0.615385  0.605128  0.569231  0.476923  0.369231  0.471795   
9     10  0.723077  0.530769  0.484615  0.438462  0.300000  0.384615   
10    31  0.692308  0.600000  0.484615  0.369231  0.353846  0.423077   
11    31  0.641026  0.553846  0.502564  0.523077  0.317949  0.410256   
12    

walk:round_trip_walk_3,phone:sharp025,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0      1  0.526316  0.798246  0.482456  0.517544  0.307018  0.421053   
1      2  0.502924  0.719298  0.385965  0.403509  0.298246  0.374269   
2      3  0.538012  0.678363  0.333333  0.391813  0.315789  0.327485   
3      4  0.461988  0.614035  0.444444  0.520468  0.228070  0.397661   
4      5  0.543860  0.657895  0.403509  0.438596  0.289474  0.342105   
5      6  0.578947  0.631579  0.552632  0.412281  0.315789  0.368421   
6      7  0.491228  0.567251  0.409357  0.426901  0.339181  0.339181   
7      8  0.578947  0.482456  0.464912  0.421053  0.087719  0.298246   
8      9  0.543860  0.535088  0.412281  0.456140  0.166667  0.280702   
9     10  0.409357  0.432749  0.497076  0.339181  0.251462  0.356725   
10    31  0.736842  0.605263  0.596491  0.447368  0.263158  0.280702   
11    31  0.587719  0.535088  0.359649  0.342105  0.315789  0.245614   
12  

walk:round_trip_walk_4,phone:hTCU11,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0     11  0.469231  0.438462  0.400000  0.784615  0.423077  0.553846   
1     12  0.405128  0.687179  0.605128  0.743590  0.405128  0.589744   
2     13  0.400000  0.676923  0.569231  0.707692  0.423077  0.538462   
3     14  0.430769  0.592308  0.600000  0.730769  0.446154  0.438462   
4     15  0.523077  0.569231  0.553846  0.707692  0.453846  0.484615   
5     16  0.461538  0.538462  0.700000  0.600000  0.330769  0.515385   
6     17  0.600000  0.515385  0.600000  0.476923  0.415385  0.361538   
7     18  0.600000  0.630769  0.546154  0.500000  0.376923  0.538462   
8     19  0.692308  0.376923  0.615385  0.630769  0.507692  0.330769   
9     20  0.553846  0.353846  0.723077  0.584615  0.461538  0.430769   
10    36  0.446154  0.507692  0.715385  0.576923  0.492308  0.507692   
11    36  0.723077  0.492308  0.630769  0.507692  0.492308  0.392308   
12    

walk:round_trip_walk_4,phone:sharp025,wireless_fingerprint:
   label  Beacon_1  Beacon_2  Beacon_3  Beacon_4  Beacon_5  Beacon_6  \
0     11  0.394737  0.368421  0.491228  0.842105  0.333333  0.482456   
1     12  0.578947  0.345029  0.385965  0.748538  0.345029  0.485380   
2     13  0.543860  0.596491  0.526316  0.543860  0.403509  0.403509   
3     14  0.412281  0.491228  0.552632  0.578947  0.298246  0.368421   
4     15  0.350877  0.532164  0.590643  0.637427  0.374269  0.415205   
5     16  0.438596  0.333333  0.491228  0.692982  0.280702  0.333333   
6     17  0.456140  0.385965  0.622807  0.657895  0.456140  0.315789   
7     18  0.684211  0.403509  0.438596  0.640351  0.359649  0.368421   
8     19  0.508772  0.502924  0.573099  0.567251  0.385965  0.374269   
9     20  0.491228  0.412281  0.675439  0.438596  0.280702  0.403509   
10    36  0.561404  0.456140  0.596491  0.570175  0.482456  0.307018   
11    36  0.596491  0.397661  0.701754  0.549708  0.385965  0.391813   
12  